In [1]:
import os
import sys
import pprint
import inspect
# Get current working directory instead of __file__
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
from modelFunc import setAndTestPlotModel, setAndTestPlotModel_with_extra_input, setAndTestModelByDate, run_test_and_save_results, run_test_and_save_results_with_extra_input
# model
from ModelArchitecture.ConvNeXtTinyModel import ConvNeXtTinyWheatModelWithConfidence
from ModelArchitecture.DenseNetModel import DenseNet121WheatModel
from ModelArchitecture.EfficientNetV2Model import EfficientNetV2SWheatCountWithConfidence
from ModelArchitecture.RepVGGA1Model import RepVGGA1WheatModelWithConfidence
from ModelArchitecture.EfficientNetV2MModel import EfficientNetV2MWheatModelWithConfidence
from ModelArchitecture.RepVGGB1g4Model import RepVGGB1g4Model
from ModelArchitecture.RegNetY8GFModel import RegNetY8GFModel
from ModelArchitecture.MobileViTV2Model import MobileViTV2WheatModel
from ModelArchitecture.MobileViTV2AddoneextrainputModel import MobileViTV2ConfidenceAddoneextrainput, MobileViTV2ConfidenceAddtwoextrainput
from ModelArchitecture.ConvNeXtTinyAddoneextrainputModel import ConvNeXtTinyConfidenceAddoneextrainput, ConvNeXtTinyConfidenceAddtwoextrainput
from ModelArchitecture.EfficientNetV2SAddextrainputModel import EfficientNetV2SConfidenceAddoneextrainput, EfficientNetV2SConfidenceAdd2Inputs
from ModelArchitecture.DenseNetAddextrainputModel import DenseNetConfidenceAddoneextrainput, DenseNetConfidenceAdd2Inputs
from ModelArchitecture.RepVGGA1AddextrainputModel import RepVGGA1ConfidenceAddoneextrainput, RepVGGA1ConfidenceAdd2Inputs
from ModelArchitecture.EfficientNetV2MAddextrainputModel import EfficientNetV2MConfidenceAddeonextrainput, EfficientNetV2MConfidenceAdd2Inputs
from ModelArchitecture.RegNetY8GFAddextrainputModel import RegNetY8GFConfidenceAddoneextrainput, RegNetYConfidenceAdd2Inputs
from ModelArchitecture.MobileOneModel import MobileOneConfidence, MobileOneConfidenceAddoneextrainput, MobileOneConfidenceAddtwoextrainput
from ModelArchitecture.MaxViTModel import MaxViTConfidenceModel, MaxViTConfidenceAddoneextrainput, MaxViTConfidenceAddtwoextrainput
from ModelArchitecture.CoatNetModel import CoAtNetConfidenceModel, CoAtNetConfidenceAddOneExtraInput, CoAtNetConfidenceAddTwoExtraInput
from ModelArchitecture.FocalNetModel import FocalNetConfidenceModel, FocalNetConfidenceAddoneextrainput, FocalNetConfidenceAddtwoextrainput
from ModelArchitecture.SwinTransformerV2Model import SwinV2ConfidenceModel, SwinV2ConfidenceAddOneExtraInput, SwinV2ConfidenceAddTwoExtraInput
from ModelArchitecture.EfficientFormerV2Model import EfficientFormerConfidenceModel, EfficientFormerConfidenceAddOneExtraInput, EfficientFormerConfidenceAddTwoExtraInput
from ModelArchitecture.PoolFormerModel import PoolFormerConfidenceModel, PoolFormerConfidenceAddOneExtraInput, PoolFormerConfidenceAddTwoExtraInput
from ModelArchitecture.ConvNeXtV2Model import ConvNeXtV2WheatModelWithConfidence, ConvNeXtV2ConfidenceAddoneextrainput, ConvNeXtV2ConfidenceAddtwoextrainput

/Users/ice/.pyenv/versions/3.10.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# run_test_and_save_results( "./allTestLabel/allResultData_raw1_day13.csv", "20StrawWeightAfterDry", ConvNeXtTinyWheatModelWithConfidence, "./allModel/20StrawWeightAfterDry_ConvNeXtTinyWheatModelWithConfidence_raw1.pth" )
# run_test_and_save_results( "./allTestLabel/allResultData_raw1_day13.csv", "20StrawWeightAfterDry", DenseNet121WheatModel, "./allModel/20StrawWeightAfterDry_DenseNet121WheatModel_raw1.pth" )
# run_test_and_save_results( "./allTestLabel/allResultData_raw1_day13.csv", "20StrawWeightAfterDry", EfficientNetV2MWheatModelWithConfidence, "./allModel/20StrawWeightAfterDry_EfficientNetV2MWheatModelWithConfidence_raw1.pth" )
# run_test_and_save_results( "./allTestLabel/allResultData_raw1_day13.csv", "20StrawWeightAfterDry", MobileViTV2WheatModel, "./allModel/20StrawWeightAfterDry_MobileViTV2WheatModel_raw1.pth" )
# run_test_and_save_results( "./allTestLabel/allResultData_raw1_day13.csv", "20StrawWeightAfterDry", RegNetY8GFModel, "./allModel/20StrawWeightAfterDry_RegNetY8GFModel_raw1.pth" )

In [2]:
def get_imported_class_dict():
    current_module = sys.modules[__name__]
    return {
        name: obj
        for name, obj in globals().items()
        if inspect.isclass(obj) and obj.__module__ != current_module.__name__
    }
modelDict = get_imported_class_dict()

In [3]:
def checkIfWeeklyData(target, extraInpList):
    weeklyData = ["Height", "LAI", "SPAD", "leafWidth", "leafLength"]
    if(target in weeklyData):
        for input in extraInpList:
            if(input in weeklyData):
                return True
        return True
    return False

def checkIfResultData(target):
    resultData =['totEarWeight','totEarNum','avgEarSize',
                 '20StrawWeightBeforeDry','20StrawWeightAfterDry',
                 'totalSeedNum','totalSeedWeightBeforeDry','totalSeedWeightAfterDry']
    if(target in resultData):
        return True
    return False

def checkIfhasExtra(name):
    extraInpList = []
    for compo in name.split("_"):
        if(compo[:5] == "extra"):
            extraInpList = compo.split("-")[1:]
            if(extraInpList[-1][-4:]== ".pth"):
                extraInpList[-1] = extraInpList[-1][:-4]

    return extraInpList

def checkIfRaw1Data(name):
    if(name.split("_")[-1].split(".")[0] == "raw1"):
        return True
    return False

modelFilePath = "./allModel/"

bb = []
for fullModelName in sorted(os.listdir(modelFilePath)):
    if( fullModelName == ".gitkeep" ):
        continue
    isExtra = False
    modelPath = modelFilePath + fullModelName
    modelTarget = fullModelName.split("_")[0]
    modelBackbone = fullModelName.split("_")[1].split('.')[0]

    if(modelTarget == "days"):
        dayData = fullModelName.split("_")[-1].split(".")[0]
        if( dayData == "noERR" ):
            datafile = "dayData_raw1_augment6_noErrDateAll.csv"
        elif( dayData == "from3" ):
            datafile = "dayData_raw1_augment6_noErrDateFrom3.csv"
        elif( dayData == "from4" ):
            datafile = "dayData_raw1_augment6_noErrDateFrom4.csv"
        dataPath = "./allTestLabel/" + datafile

        print("modelPath: ", modelPath)
        print("dataPath: ", dataPath)
        print("modelTarget: ", modelTarget)
        print("modelBackbone: ", modelBackbone)
        run_test_and_save_results(dataPath, modelTarget, modelDict[modelBackbone], modelPath)
        continue

    extraInpList = checkIfhasExtra(fullModelName)
    isWeeklyData = checkIfWeeklyData(modelTarget, extraInpList)
    isRaw1Data = checkIfRaw1Data(fullModelName)

    if(isWeeklyData and isRaw1Data):
        datafile = "allWeeklyData_raw1_day13.csv"
    elif(isWeeklyData and not isRaw1Data):
        datafile = "allWeeklyData_raw3_day13.csv"
    elif(not isWeeklyData and isRaw1Data):
        datafile = "allResultData_raw1_day13.csv"
    else:
        datafile = "allResultData_raw3_day13.csv"
    dataPath = "./allTestLabel/" + datafile

    print("modelPath: ", modelPath)
    print("dataPath: ", dataPath)
    print("modelTarget: ", modelTarget)
    print("modelBackbone: ", modelBackbone)
    if(extraInpList!=[]):
        print("extraInput: ", extraInpList)
        run_test_and_save_results_with_extra_input(dataPath, modelTarget, modelDict[modelBackbone], modelPath, extraInpList)
    else:
        run_test_and_save_results(dataPath, modelTarget, modelDict[modelBackbone], modelPath)

modelPath:  ./allModel/20StrawWeightAfterDry_ConvNeXtTinyWheatModelWithConfidence_raw1.pth
dataPath:  ./allTestLabel/allResultData_raw1_day13.csv
modelTarget:  20StrawWeightAfterDry
modelBackbone:  ConvNeXtTinyWheatModelWithConfidence
Using device: mps


/Users/ice/.pyenv/versions/3.10.10/lib/python3.10/site-packages/torch/__init__.py:1236: UserWarning: torch.set_default_tensor_type() is deprecated as of PyTorch 2.1, please use torch.set_default_dtype() and torch.set_default_device() as alternatives. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/tensor/python_tensor.cpp:436.)
  _C._set_default_tensor_type(t)


✅ Safe Split → Train: 23920, Val: 2980, Test: 150
Train Batches: 1495, Validation Batches: 187, Test Batches: 10


Testing:   0%|          | 0/10 [00:02<?, ?it/s]


FileNotFoundError: Caught FileNotFoundError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/Users/ice/.pyenv/versions/3.10.10/lib/python3.10/site-packages/torch/utils/data/_utils/worker.py", line 349, in _worker_loop
    data = fetcher.fetch(index)  # type: ignore[possibly-undefined]
  File "/Users/ice/.pyenv/versions/3.10.10/lib/python3.10/site-packages/torch/utils/data/_utils/fetch.py", line 52, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/Users/ice/.pyenv/versions/3.10.10/lib/python3.10/site-packages/torch/utils/data/_utils/fetch.py", line 52, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/Volumes/PortableSSD/MasterProj/Model_Creation/TraitPredictionModel/dataClass.py", line 34, in __getitem__
    rgb = resize_rgb_return_array(rgb_path, target_size=(512, 256)) / 255.0
  File "/Volumes/PortableSSD/MasterProj/Model_Creation/TraitPredictionModel/rgbdsmAlignment.py", line 